## 计算机如何存储数字
计算机中存储的都是补码，对于32位整型数字，存储范围: $[-2^{31}, 2^{31}-1]$

**原码：**
- 最高位作为符号位，0表示正数，1表示负数。

**反码：**
- 正数的反码与原码相同；
- 负数的反码，**符号位不变**，其余位0变1, 1变0。

**补码：**
- 正数的补码与原码相同；
- 负数的补码，**符号位不变**，取反码+1；
- 特殊补码（以8位为例）：`1000 0000` 表示-128。
    - 对于数字 `-0`  
    - 原码：`1000 0000`  
    - 反码：`1111 1111`  
    - 补码：`1000 0000`  
    - 但是现实中，我们指导 `-0` 与 `0` 其实并没有差别, 既然已经有补码`0000 0000`来表示0了，那么就把多出来的补码`1000 0000`用来表示$-2^n$，也就是`-128`。
    - 由于-128无法通过原码转换而来，所以这个数字是没有原码和反码的。

**示例：**

| 十进制 | 二进制原码 | 二进制反码 | 二进制补码 |
| ---- | ---- | ---- | ---- |
| 1 | `0000 0001` | `0000 0001` | `0000 0001` |
| -1 | `1000 0001` | `1111 1110` | `1111 1111` |
| 0 | `0000 0000` | `0000 0000` | `0000 0000` |
| -128 | 无 | 无 | `1000 0000` |


## 为什么是补码

**题目:**
- 计算 `1 + (-1)`的值  

**原码计算:**  
$$
\begin{split}
&0000\ 0001\\
+\ &1000\ 0001\\
\end{split}
\over {=\ 1000\ 0010}
$$  
结果 `1000 0010 = -2`,这明显与预期结果是不符合的。为了解决这个问题，出现了反码。  

**反码计算:**  
$$
\begin{split}
&0000\ 0001\\
+\ &1111\ 1110\\
\end{split}
\over {=\ 1111\ 1111}
$$  
结果反码 `1111 1111` 的原码 `1000 0000 = -0`, 这个值的真实含义与预期结果一致。但是在反码中 `0000 0000`, `1000 0000` 两个反码均可以表示数字`0`。为了解决这个问题，出现了补码。

**补码计算:**  
$$
\begin{split}
&0000\ 0001\\
+\ &1111\ 1111\\
\end{split}
\over {=\ 0000\ 0000}
$$  
结果补码 `0000 0000` 的原码 `0000 0000 = 0`, 与预期结果一致。

## 算术运算  
**加法：**  
先来看看十进制如何进行加法运算的，例如：`13 + 18 = 31`
- 定义两个过程函数：
    - $sum(a,b): $不考虑进位的各位相加, 例如 `13 + 18 = 21`
    - $carry(a,b)：$只考虑进位的各位相加, 例如`13 + 18 = 010`
- 步骤一：计算$sum(a,b)$
- 步骤二：计算$carry(a,b)$
- 步骤三: 检查步骤二的结果
    - 步骤二结果等于0，那么结束运算，步骤一的运算结果即为最终的答案
    - 步骤二结果不等于0，那么令a=步骤一结果，b=步骤二结果，重新执行步骤一、二、三

| a | b | sum(a,b) | carry(a,b) |
| ---- | ---- | ---- | ---- |
| 53 | 58 | 1 | 110 |
| 1 | 110 | 111 | 0 |

两轮运算后，carry(a,b)的值已经是0，所以计算结果就是31

将办法运用到二进制中：
- 13的二进制：`0000 1101`
- 18的二进制：`0001 0010`  

| a, b | sum(a,b) | carry(a,b) |
| ---- | ---- | ---- |
| $$\begin{split}a: 0000\ 1101\\b: 0001\ 0010\end{split}$$ | $0001\ 1111$ | $0000\ 0000$ |

一轮运算后，carry(a,b)的值已经是0，所以计算结果就是: `0001 1111 = 31`。  

其实在二进制中有：
- $sum(a,b)\ =\ a\ \^\ b$
- $carry(a,b)\ =\ (a\ \&\ b)\ <<\ 1$
- $a + b = (a\ \^\ b) + ((a\ \&\ b) << 1)$

In [1]:
"""
python中的数字存储并不是限制32位的, 为了模拟32位integer的运算, 需要对数字进行一些特殊处理
1. 对每个数高位去除处理: &= mask_1
2. 对负数结果高位去除处理: ~((a ^ mask_2) ^ mask_3)
"""
def plus(a, b):
    mask_1, mask_2, mask_3 = 0xFFFFFFFF, 0x80000000, 0x7FFFFFFF
    a &= mask_1
    b &= mask_1
    while b:
        a, b = (a ^ b) & mask_1, ((a & b) << 1) & mask_1
    if a & mask_2:
        return ~((a ^ mask_2) ^ mask_3)
    return a

plus(13, 18)

31

**减法：**

其实计算机在进行减法操作的时候，进行了一个巧妙的变化。我们都知道`11 - 13`可以转化为`11 + (-13)`,也就是把减法变成了加法，将减号运算符转移到了被减数的符号属性上。然后再利用加法即可得到结果。  
- 11: 原码=`0000 1011`，反码=`0000 1011`，补码=`0000 1011`
- -13：原码=`1000 1101`，反码=`1111 0010`，补码=`1111 0011`

| a, b   | a ^ b | (a & b) << 1 |
| ---- | ---- | ---- |
| $$\begin{split}a: 0000\ 1011\\b: 1111\ 0011\end{split}$$ | $1111\ 1000$ | $0000\ 0110$ |
| $$\begin{split}a: 1111\ 1000\\b: 0000\ 0110\end{split}$$ | $1111\ 1110$ | $0000\ 0000$ |

结果`1111 1110`，反码：`1000 0001`，补码：`1000 0010 = -2`，最终结果符合预期

In [2]:
def subtract(a, b):
    return a + (-b)
    
subtract(11, 13)

-2

**乘法：**

首先看看十进制是如何完成乘法的，计算一下`11 * 13`:  
- 计算个位： `11 * 3 = 33`
- 计算十位： `11 * 10 = 110`
- 计算和： `33 + 110 = 143`  
- 故： `11 * 33 = 143`

将十进制的计算办法移植到二进制的世界中应该也是一样的过程：
- 参数一：`11 = 0000 1011`
- 参数二：`13 = 0000 1101`
- 从低到高依次计算  
    - `0000 1011 * 0000 0001 = 0000 1011 << 0 = 0000 1011`
    - `0000 1011 * 0000 0000 = 0000 0000`
    - `0000 1011 * 0000 0100 = 0000 1011 << 2 = 0010 1100`
    - `0000 1011 * 0000 1000 = 0000 1011 << 3 = 0101 1000`
- 对所有分步结果求和
    - `0000 1011 + 0010 1100 = 0011 0111`
    - `0011 0111 + 0101 1000 = 1000 1111`
- 结果转换 `0110 1111 = 1 + 2 + 4 + 8 + 128 = 143`

In [3]:
def multiply(a, b):
    rev = (a ^ b) < 0
    a, b = -a if a < 0 else a, -b if b < 0 else b
    ans = 0
    while b:
        if b & 1:
            ans += a
        a <<= 1
        b >>= 1
    return -ans if rev else ans

multiply(11, 13)

143

**除法：**

除法比较难处理，普通想法就是被除数不断减除数，直到不够减。但是减法涉及到了高位借位操作，尝试是否可以通过加法来完成。

让我们来看一个例子：`145 / 13 = 11`
- 递增尝试最大除数
    - `13 << 0 = 13`，小于145，可以除尽
    - `13 << 1 = 26`，小于145，可以除尽
    - `13 << 2 = 52`，小于145，可以除尽
    - `13 << 3 = 104`，小于145，可以除尽
    - `13 << 4 = 208`，大于145，不能除尽
- 定义两个变量，`sum = 0, ans = 0`
- 从大到小利用加法逐步接近被除数
    - `sum + 13 << 3 = 0 + 104 < 145, sum += 104, ans = ans | (1 << 3) = ob1000 = 8`
    - `sum + 13 << 2 = 104 + 52 = 156 > 145`, sum 不变, ans 不变  
    - `sum + 13 << 1 = 104 + 26 = 130 < 145, sum += 26, ans = ans | (1 << 1) = ob1010 = 10`  
    - `sum + 13 << 0 = 130 + 13 = 143 < 145, sum += 13, ans = ans | (1 << 0) = ob1011 = 11`
- 那么ans的最后值就是原问题的解  

可以看到, 整个过程分为两大步骤:
- 第一步求得最大n使得 $13 << n$的值小于被除数145。
- 第二步从大到小尝试累加$13 << n$，并排除累加后的值大于被除数145的n

实际中，可能会省略掉第一步，直接假设n=31(int类型的数字使用32位进行存储)，然后利用步骤二得到解。

In [4]:
def divide(a, b):
    rev = (a ^ b) < 0
    a, b = -a if a < 0 else a, -b if b < 0 else b
    ans = 0
    for i in range(31, -1, -1):
        step = b << i
        if step and a - step >= 0:
            ans |= 1 << i
            a -= step
    return -ans if rev else ans

divide(143, 13)

11

## 经典应用

**二进制中1的个数:**

- 有 `a = 5 = 0000 0101, a - 1 = 4 = 0000 0100`,  
- 那么 `a & (a - 1) = 0000 0100` 则刚好可以去除a的最右边一个1
- 利用以上规则，代码可以这样写：  
    ```
    def numOfOne(a):
        if a == 0 or a == 1:
            return a
        num = 0
        while a:
            num += 1
            a = a & (a - 1)
        return num
    ```

**不额外增加空间交换两个数的值:**

- 如果 `a ^ b = c`  
- 那么 `b ^ c = a, a ^ c = b`
- 利用以上规则，代码可以这样写：  
    ```
    def exchange(a, b):
        a ^= b
        b ^= a
        a ^= b
    ```
- 解释一下，三个步骤:
    - 第一步: `c = a ^ b`
    - 第二步: `b = c ^ b = a`
    - 第三步: `a = c ^ a = b`


**奇偶判断：**

- 一个数是否是奇数，只需要看二进制最低位是否是1
- 利用以上规则，代码可以这样写：  
    ```
    def isOdd(a):
        return a & 1 == 1
    ```


**两个数平均数：**

```
def avg(a, b):
    return (a & b) + ((a ^ b) >> 1)
```


**其他快捷用法：**
- 对于二进制数字`0b110100`，想要获取到低位数字`0b100`，可以用 `a & (-a)`  

- 快速消除最低位1：对于二进制数字`0b1101`，想要消除最低位的1得到`0b1100`，可以用  `a & (a - 1)`